In [1]:
from pandas_ml_utils import pd


## Fist lets load some data

In [2]:
df = pd.read_csv('SPY.csv')
df.tail()


,Date,Open,High,Low,Close,Adj Close,Volume
6758,2019-11-29,314.859985,315.130005,314.059998,314.309998,314.309998,36592700
6759,2019-12-02,314.589996,314.660004,311.170013,311.640015,311.640015,75767800
6760,2019-12-03,308.649994,309.640015,307.130005,309.549988,309.549988,73941700
6761,2019-12-04,310.700012,312.119995,310.320007,311.459991,311.459991,49080000
6762,2019-12-05,312.230011,312.250000,310.580109,312.089996,312.089996,40781669


## Fit Regularized Model
Now we just build a very naive model where we add an L1 and L2 penalization term. Don't expect the model to predict
anything useful

In [3]:
model_name = '/tmp/regression_with_regularization.model'

# in order to restore from a gobal namespace you need to wrap everything in a function
def train_model():
    with df.model(model_name) as m:
        from pandas_ml_common.utils.column_lagging_utils import lag_columns
        from pandas_ml_utils import FeaturesAndLabels, RegressionSummary, FittingParameter
        from pandas_ml_utils_torch import PytorchModel
        from pandas_ml_utils_torch.merging_cross_folds import take_the_best
        from torch import nn

        def net_provider():
            from pandas_ml_utils_torch import PytorchNN
            from torch import nn

            class Net(PytorchNN):

                def __init__(self):
                    super().__init__()
                    self.net = nn.Sequential(
                        nn.Linear(10, 4),
                        nn.Tanh(),
                        nn.Linear(4, 4),
                        nn.Tanh(),
                        nn.Linear(4, 1),
                        nn.Tanh(),
                    )

                def L1(self):
                    # path to the parameters which should be regularized
                    # the path is constructed from self.named_parameters() and allows the use of wildcards
                    return {'net/0/**/weight': 0.02}

                def L2(self):
                    return {
                        'net/0/**/weight': 0.02,
                        'net/2/**/weight': 0.05
                    }

                def forward_training(self, x):
                    return self.net(x)

            return Net()

        def loss(params):
            from torch.optim import SGD
            return SGD(params, lr=0.01, momentum=0.0)

        fit = m.fit(
            PytorchModel(
                net_provider,
                FeaturesAndLabels(
                    [lambda df: lag_columns(df["Close"].pct_change(), range(10))],
                    [lambda df: df["Close"].pct_change().shift(-1)]),
                nn.MSELoss,
                loss,
                merge_cross_folds=take_the_best,
                summary_provider=RegressionSummary
            ),
            FittingParameter(epochs=100),
            verbose=1
        )

    return fit

train_model()

epoch: 0, train loss: 0.3068681061267853, test loss: 0.3066888153553009
epoch: 1, train loss: 0.2997573912143707, test loss: 0.29958096146583557
epoch: 2, train loss: 0.292947381734848, test loss: 0.2927737832069397
epoch: 3, train loss: 0.2864375114440918, test loss: 0.28626662492752075
epoch: 4, train loss: 0.2802257835865021, test loss: 0.2800576090812683
epoch: 5, train loss: 0.27430903911590576, test loss: 0.2741434574127197
epoch: 6, train loss: 0.2686828374862671, test loss: 0.2685198187828064
epoch: 7, train loss: 0.26334184408187866, test loss: 0.26318132877349854
epoch: 8, train loss: 0.2582796514034271, test loss: 0.258121520280838
epoch: 9, train loss: 0.2534891366958618, test loss: 0.25333333015441895
epoch: 10, train loss: 0.2489624321460724, test loss: 0.24880895018577576
epoch: 11, train loss: 0.2446911334991455, test loss: 0.24453985691070557
epoch: 12, train loss: 0.24066630005836487, test loss: 0.24051718413829803
epoch: 13, train loss: 0.23687872290611267, test loss

/home/kic/.local/lib/python3.8/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
mean_gamma_deviance failed: (ValueError('Mean Tweedie deviance error with power=2 can only be used on strictly positive y and y_pred.'), 'error while calling <function mean_gamma_deviance at 0x7fbcecc14430>(FullArgSpec(args=[], varargs=\'args\', varkw=\'kwargs\', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={}))\n@_deprecate_positional_args\ndef mean_gamma_deviance(y_true, y_pred, *, sample_weight=None):\n    """Mean Gamma deviance regression loss.\n\n    Gamma deviance is equivalent to the Tweedie deviance with\n    the power parameter `power=2`. It is invariant to scaling of\n    the target variable, and measures relative errors.\n\n    Read more in the :ref:`User Guide <mean_tweedie_deviance>`.\n\n    P

In [4]:
import time
time.sleep(0.5)